## import

In [1]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from utils import *
import cv2
from sklearn.model_selection import StratifiedKFold
import zipfile

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# Pretrained Model
import segmentation_models_pytorch as smp

# torchvision Models
import torchvision
from torchvision import models
from torchvision.models.segmentation.deeplabv3 import DeepLabHead

# 전처리를 위한 라이브러리
from torch.utils.data.sampler import WeightedRandomSampler
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

plt.rcParams['axes.grid'] = False

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장
%matplotlib inline

pytorch version: 1.5.0+cu101
GPU 사용 가능 여부: True
Tesla V100-PCIE-32GB
1


## 하이퍼파라미터 세팅 및 seed 고정

In [2]:
batch_size = 20  # Mini-batch size
num_epochs = 40
learning_rate = 1e-4

In [3]:
# seed 고정
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
# torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [4]:
class CustomDataLoader(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'test', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join('../input/data', image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            
            return images, image_infos
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

## Dataset 정의 및 DataLoader 할당

In [5]:
# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

test_transform = A.Compose([
                            A.Normalize(
                                mean=(0.485, 0.456, 0.406),
                                std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0
                            ),
                            ToTensorV2(),
                           ])

In [6]:
test_dataset = CustomDataLoader('../input/data/test.json', mode="test", transform=test_transform)

test_loader = DataLoader(test_dataset, 
                          batch_size=1, 
                          num_workers=4,
                         shuffle=False,
                         collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


## Inference Load Model

In [13]:
MODEL_PATHS = ['./saved/0_checkpoint.pt',
                './saved/1_checkpoint.pt',
                './saved/2_checkpoint.pt',
                './saved/3_checkpoint.pt',
                './saved/4_checkpoint.pt']

OUT_MASKS = f'./saved/test_masks.zip'

In [14]:
def inference(models, imgs, device):
    outs = None
    flip_outs = None
    flips = [[-1],[-2],[-2,-1]]
    for model in models:
        model.eval()
        if outs == None:
            outs = model(imgs.to(device).float()).detach()
        else:
            outs += model(imgs.to(device).float()).detach()
        
    outs /= len(models)
        
    for flip in flips:
        flip_img = torch.flip(imgs, flip)
        tmp_outs = None
        for model in models:
            flip_out = model(flip_img.to(device).float()).detach()
            flip_out = torch.flip(flip_out, flip)
            if tmp_outs == None:
                tmp_outs = flip_out
            else:
                tmp_outs += flip_out
        tmp_outs /= len(models)
        if flip_outs == None:
            flip_outs = tmp_outs
        else:
            flip_outs += tmp_outs
    flip_outs /= 3
    
    outs += flip_outs
    
    return outs/2

## Inference

In [15]:
def test(models, data_loader, device):
    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction.')
    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)
    pbar = tqdm(enumerate(data_loader), total=len(data_loader), position=0, leave=True)
    with torch.no_grad():
        with zipfile.ZipFile(OUT_MASKS, 'w') as mask_out:
            for step, (imgs, image_infos) in pbar:
                # print(imgs)
                # print(imgs)
                imgs = torch.stack(imgs)

                outs = inference(models, imgs, device)

                oms = torch.argmax(outs, dim=1).detach().cpu().numpy()
                
                file_name = image_infos[0]['file_name'].split("/")
                file_name[0] += "_masks"
                file_name[1] = file_name[1][:-4]
                file_name = "/".join(file_name)

                m = cv2.imencode(".png", oms.squeeze())[1]
                mask_out.writestr(f"{file_name}.png", m)

                # resize (256 x 256)
                temp_mask = []
                for img, mask in zip(np.stack(imgs), oms):
                    # print(mask.shape)
                    transformed = transform(image=img, mask=mask)
                    mask = transformed['mask']
                    temp_mask.append(mask)

                oms = np.array(temp_mask)
                oms = oms.reshape([oms.shape[0], size*size]).astype(int)

                preds_array = np.vstack((preds_array, oms))

                file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    return file_names, preds_array

In [16]:
models=[]
for path in MODEL_PATHS:
    model = smp.DeepLabV3Plus('resnext50_32x4d', encoder_weights=None, classes=12)
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint)
    model.eval()
    model.to(device)
    models.append(model)
    
    del checkpoint

In [17]:
file_names, preds = test(models, test_loader, device)

Start prediction.


  0%|          | 0/837 [00:00<?, ?it/s]

End prediction.


## submission.csv 생성

In [18]:
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)

# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv("./submission/psheudo_mixmatch.csv", index=False)

## Reference

